In [2]:
import numpy as np, pandas as pd

def mpPDF(var, q, pts):
    #Marcenko-pastur pdf
    #q=T/N
    eMin, eMax = var*(1-(1./q)**.5)**2, var*(1+(1./q)**.5)**2
    eVal = np.linspace(eMin, eMax, pts)
    pdf = q/(2*np.pi*var*eVal)*((eMax-eVal)*(eVal-eMin))**.5
    pdf = pd.Series(pdf, index=eVal)

    return pdf



In [3]:
from sklearn.neighbors import KernelDensity

def getPCA(matrix):
    #에르미트 행렬(Hermitian matrix)로 부터 eVal, eVec을 얻는다
    eVal, eVec = np.linalg.eigh(matrix)
    indices=eVal.argsort()[::-1] #eval을 내림차순으로 정렬하고 그 인덱스를 추출한다.
    eVal, eVec = eVal[indices], eVec[:, indices]
    eVal = np.diagflat(eVal)
    return eVal, eVec

def fitKDE(obs, bwidth=.25, kernel='gaussian', x=None):
    #커널을 관측시리즈에 적합화하고, 관측확률을 도출한다.
    # x는 kde가 평가된 값의 배열이다.
    if len(obs.shape)==1:obs=obs.reshape(-1,1)
    kde = KernelDensity(kernel=kernel, bandwidth=bwidth).fit(obs)
    if x is None:x=np.unique(obs).reshape(-1, 1)
    if len(x.shape)==1 : x=x.reshape(-1,1)
    logProb=kde.score_samples(x) #log(density)
    pdf = pd.Series(np.exp(logProb), index=x.flatten())
    return pdf

x = np.random.normal(size=(10000, 1000))
eVal0, eVec0 = getPCA(np.corrcoef(x,rowvar=0))
pdf0 = mpPDF(1., q=x.shape[0]/float(x.shape[1]), pts=1000)
pdf1 = fitKDE(np.diag(eVal0), bwidth=.01)#empirical pdf
    

In [ ]:
pdf0